In [5]:
import duckdb
import pandas as pd
import pyarrow.parquet as pq

In [6]:
conn = duckdb.connect(database=':memory:', read_only=True)

CatalogException: Catalog Error: Cannot launch in-memory database in read-only mode!

In [4]:
df = pd.DataFrame()

In [ ]:
for file in files:
    df = df.append(pq.read_table(file).to_pandas())